In [1]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])


sc = SparkContext(conf=conf).getOrCreate()
ss = SparkSession(sc).builder.config("spark.driver.memory", "6g").getOrCreate()

import pyspark.sql.functions as f
#import pyspark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/14 16:53:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [35]:
#import multiprocessing

#multiprocessing.cpu_count()

8

In [2]:
ss

In [3]:
#4.930
#0.811

In [4]:
import time

start = time.time()
weather_df = ss.read.option('header', True).csv("weather-ta2.csv").limit(20000).repartition('station_id')
print(time.time() - start)
weather_df.show()

4.944303035736084


22/03/14 16:54:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , station_id, date, tmp_val, dew_val
 Schema: _c0, station_id, date, tmp_val, dew_val
Expected: _c0 but found: 
CSV file: file:///Users/nk_dan/Desktop/weather-ta2.csv


+-----+-----------+----------+-------+-------+
|  _c0| station_id|      date|tmp_val|dew_val|
+-----+-----------+----------+-------+-------+
|10000|08524099999|2016-01-01|    175|    147|
|10001|08524099999|2016-01-02|    174|    159|
|10002|08524099999|2016-01-03|    182|    160|
|10003|08524099999|2016-01-04|    177|    162|
|10004|08524099999|2016-01-05|    170|    167|
|10005|08524099999|2016-01-06|    149|     76|
|10006|08524099999|2016-01-07|    139|    101|
|10007|08524099999|2016-01-08|    160|    138|
|10008|08524099999|2016-01-09|    150|    109|
|10009|08524099999|2016-01-10|    152|     92|
|10010|08524099999|2016-01-11|    174|    158|
|10011|08524099999|2016-01-12|    177|    172|
|10012|08524099999|2016-01-13|    175|    163|
|10013|08524099999|2016-01-14|    172|    156|
|10014|08524099999|2016-01-15|    175|    153|
|10015|08524099999|2016-01-16|    179|    168|
|10016|08524099999|2016-01-17|    144|    131|
|10017|08524099999|2016-01-18|    150|    139|
|10018|085240

In [5]:
start = time.time()
stocks_df=ss.read.option('header', True).csv('stocks-ta2.csv').repartition('date')
print(time.time() - start)
stocks_df.show()

0.9762401580810547


22/03/14 16:54:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stock_name, date, price
 Schema: _c0, stock_name, date, price
Expected: _c0 but found: 
CSV file: file:///Users/nk_dan/Desktop/stocks-ta2.csv


+----+--------------------+----------+------+
| _c0|          stock_name|      date| price|
+----+--------------------+----------+------+
| 229|1004.Asien--Austr...|2016-08-17|  74.8|
| 498|1004.Asien--Austr...|2017-05-14| 66.49|
| 677|1004.Asien--Austr...|2017-12-05| 46.52|
|1229|10056.Asien--Aust...|2016-08-17| 471.0|
|1498|10056.Asien--Aust...|2017-05-14| 393.0|
|1677|10056.Asien--Aust...|2017-12-05| 342.0|
|2229|10300.Asien--Aust...|2016-08-17| 563.0|
|2498|10300.Asien--Aust...|2017-05-14| 816.0|
|2677|10300.Asien--Aust...|2017-12-05|1011.0|
|3229|10319.Asien--Aust...|2016-08-17|1536.0|
|3498|10319.Asien--Aust...|2017-05-14|1990.0|
|3677|10319.Asien--Aust...|2017-12-05|2550.0|
|4229|10349.Asien--Aust...|2016-08-17|1040.0|
|4498|10349.Asien--Aust...|2017-05-14|1327.0|
|4677|10349.Asien--Aust...|2017-12-05|2017.0|
|5229|10359.Asien--Aust...|2016-08-17|  80.0|
|5498|10359.Asien--Aust...|2017-05-14|  73.0|
|5677|10359.Asien--Aust...|2017-12-05|  66.0|
|6229|1051.Asien--Austr...|2016-08

In [6]:
weather_df = weather_df.select('station_id', 'date', 'tmp_val').cache()

-----

**The code below is slow version**

import timeit

v1 = timeit.default_timer()

weather_df.registerTempTable("df")
 
mylist = ss.sql("""
SELECT DISTINCT  u1.station_id,
    u2.station_id
FROM df as u1 JOIN df as u2 ON u1.station_id> u2.station_id;
""").collect()
 
v2 = timeit.default_timer()

print(v2-v1)

weather_df.columns

i, j = mylist[0]

from pyspark.sql.functions import col
from pyspark.sql.types import StructType

v1 = timeit.default_timer()

schema = weather_df.filter(weather_df.station_id == i).alias('a').join(weather_df.filter(weather_df.station_id == j)\
                                         .withColumnRenamed('station_id', 'station_id_2')\
                                         .withColumnRenamed('date', 'date_2')\
                                         .withColumnRenamed('tmp_val', 'tmp_val_2').alias('b'),
                                            f.col("a.date") == f.col("b.date_2"), 'inner').schema

df = ss.createDataFrame([], schema)

for a, b in mylist:
    df = df.union(weather_df.filter(weather_df.station_id == a).alias('a').join(weather_df.filter(weather_df.station_id == b)\
                                         .withColumnRenamed('station_id', 'station_id_2')\
                                         .withColumnRenamed('date', 'date_2')\
                                         .withColumnRenamed('tmp_val', 'tmp_val_2').alias('b'),
                                            f.col("a.date") == f.col("b.date_2"), 'inner'))
    
df.show()

v2 = timeit.default_timer()
print(v2-v1)

----

----

This code is faster version

In [7]:
#0.363
#

In [8]:
#start = time.time()

df = weather_df.alias('a').join(weather_df.withColumnRenamed('station_id', 'station_id_2')\
                                          .withColumnRenamed('date', 'date_2')\
                                          .withColumnRenamed('tmp_val', 'tmp_val_2').alias('b'),
                                            f.col("a.date") == f.col("b.date_2"), 'inner').filter('station_id > station_id_2')

#print(time.time() - start)

df.show()


+-----------+----------+-------+------------+----------+---------+
| station_id|      date|tmp_val|station_id_2|    date_2|tmp_val_2|
+-----------+----------+-------+------------+----------+---------+
|08524099999|2016-01-01|    175| 02741099999|2016-01-01|       22|
|08524099999|2016-01-01|    175| 03066099999|2016-01-01|       18|
|08524099999|2016-01-01|    175| 01016099999|2016-01-01|        4|
|08524099999|2016-01-01|    175| 06207099999|2016-01-01|       79|
|08524099999|2016-01-01|    175| 04202299999|2016-01-01|     -200|
|08524099999|2016-01-01|    175| 02845099999|2016-01-01|      -32|
|08524099999|2016-01-01|    175| 06190099999|2016-01-01|       30|
|08524099999|2016-01-01|    175| 04231099999|2016-01-01|     -289|
|08524099999|2016-01-01|    175| 02520099999|2016-01-01|        6|
|08524099999|2016-01-01|    175| 03862099999|2016-01-01|       23|
|08524099999|2016-01-02|    174| 02741099999|2016-01-02|      -36|
|08524099999|2016-01-02|    174| 03066099999|2016-01-02|      

In [9]:
df.count()

190000

----

In [10]:
df.show()

+-----------+----------+-------+------------+----------+---------+
| station_id|      date|tmp_val|station_id_2|    date_2|tmp_val_2|
+-----------+----------+-------+------------+----------+---------+
|08524099999|2016-01-01|    175| 02741099999|2016-01-01|       22|
|08524099999|2016-01-01|    175| 03066099999|2016-01-01|       18|
|08524099999|2016-01-01|    175| 01016099999|2016-01-01|        4|
|08524099999|2016-01-01|    175| 06207099999|2016-01-01|       79|
|08524099999|2016-01-01|    175| 04202299999|2016-01-01|     -200|
|08524099999|2016-01-01|    175| 02845099999|2016-01-01|      -32|
|08524099999|2016-01-01|    175| 06190099999|2016-01-01|       30|
|08524099999|2016-01-01|    175| 04231099999|2016-01-01|     -289|
|08524099999|2016-01-01|    175| 02520099999|2016-01-01|        6|
|08524099999|2016-01-01|    175| 03862099999|2016-01-01|       23|
|08524099999|2016-01-02|    174| 02741099999|2016-01-02|      -36|
|08524099999|2016-01-02|    174| 03066099999|2016-01-02|      

In [11]:
df = df.select('station_id', 'station_id_2', 'date', 'tmp_val', 'tmp_val_2').cache()
weather_df.unpersist()

DataFrame[station_id: string, date: string, tmp_val: string]

In [12]:
from pyspark.sql.types import StringType, FloatType

def max(a,b):
  if float(a) > float(b):
    return float(a)
  else:
    return float(b)

def min(a,b):
  if float(a) > float(b):
    return float(b)
  else:
    return float(a)

def avg(a,b):
  return (float(a) + float(b))/2

max_udf = f.udf(max, FloatType())
min_udf = f.udf(min, FloatType())
avg_udf = f.udf(avg, FloatType())

In [14]:
start = time.time()

df = df.withColumn('min', min_udf('tmp_val', 'tmp_val_2'))\
  .withColumn('avg', avg_udf('tmp_val', 'tmp_val_2'))\
  .withColumn('max', max_udf('tmp_val', 'tmp_val_2'))

print(time.time() - start)

0.1499638557434082


In [15]:
df.show()

+-----------+------------+----------+-------+---------+------+-----+-----+
| station_id|station_id_2|      date|tmp_val|tmp_val_2|   min|  avg|  max|
+-----------+------------+----------+-------+---------+------+-----+-----+
|08524099999| 02741099999|2016-01-01|    175|       22|  22.0| 98.5|175.0|
|08524099999| 03066099999|2016-01-01|    175|       18|  18.0| 96.5|175.0|
|08524099999| 01016099999|2016-01-01|    175|        4|   4.0| 89.5|175.0|
|08524099999| 06207099999|2016-01-01|    175|       79|  79.0|127.0|175.0|
|08524099999| 04202299999|2016-01-01|    175|     -200|-200.0|-12.5|175.0|
|08524099999| 02845099999|2016-01-01|    175|      -32| -32.0| 71.5|175.0|
|08524099999| 06190099999|2016-01-01|    175|       30|  30.0|102.5|175.0|
|08524099999| 04231099999|2016-01-01|    175|     -289|-289.0|-57.0|175.0|
|08524099999| 02520099999|2016-01-01|    175|        6|   6.0| 90.5|175.0|
|08524099999| 03862099999|2016-01-01|    175|       23|  23.0| 99.0|175.0|
|08524099999| 02741099999

In [16]:
def unqiue_combination(a, b):
  col = [a, b]
  col.sort()
  return '_'.join(col)

my_udf = f.udf(unqiue_combination, StringType())

In [18]:
start = time.time()

df = df.withColumn('combo_id', my_udf('station_id', 'station_id_2')).select('combo_id', 'date', 'min', 'avg', 'max').repartition('date')

print(time.time() - start)

0.11489105224609375


In [19]:
df.show()

+--------------------+----------+------+-----+-----+
|            combo_id|      date|   min|  avg|  max|
+--------------------+----------+------+-----+-----+
|02741099999_08524...|2016-08-17| 139.0|177.5|216.0|
|03066099999_08524...|2016-08-17| 135.0|175.5|216.0|
|01016099999_08524...|2016-08-17|  32.0|124.0|216.0|
|06207099999_08524...|2016-08-17| 168.0|192.0|216.0|
|04202299999_08524...|2016-08-17|  56.0|136.0|216.0|
|02845099999_08524...|2016-08-17|  94.0|155.0|216.0|
|06190099999_08524...|2016-08-17| 150.0|183.0|216.0|
|04231099999_08524...|2016-08-17|  52.0|134.0|216.0|
|02520099999_08524...|2016-08-17| 136.0|176.0|216.0|
|03862099999_08524...|2016-08-17| 127.0|171.5|216.0|
|02741099999_08524...|2017-05-14|  -5.0| 85.0|175.0|
|03066099999_08524...|2017-05-14| 126.0|150.5|175.0|
|01016099999_08524...|2017-05-14|-125.0| 25.0|175.0|
|06207099999_08524...|2017-05-14| 115.0|145.0|175.0|
|04202299999_08524...|2017-05-14| -11.0| 82.0|175.0|
|02845099999_08524...|2017-05-14| -33.0| 71.0|

------

In [20]:
stocks_df=ss.read.option('header', True).csv('stocks-ta2.csv').limit(20000)
stocks_df.show()

22/03/14 16:54:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stock_name, date, price
 Schema: _c0, stock_name, date, price
Expected: _c0 but found: 
CSV file: file:///Users/nk_dan/Desktop/stocks-ta2.csv


+---+--------------------+----------+-----+
|_c0|          stock_name|      date|price|
+---+--------------------+----------+-----+
|  0|1004.Asien--Austr...|2016-01-01| 57.5|
|  1|1004.Asien--Austr...|2016-01-02| 57.5|
|  2|1004.Asien--Austr...|2016-01-03| 57.5|
|  3|1004.Asien--Austr...|2016-01-04| 57.8|
|  4|1004.Asien--Austr...|2016-01-05|57.17|
|  5|1004.Asien--Austr...|2016-01-06|57.65|
|  6|1004.Asien--Austr...|2016-01-07| 58.4|
|  7|1004.Asien--Austr...|2016-01-08|56.44|
|  8|1004.Asien--Austr...|2016-01-09|56.44|
|  9|1004.Asien--Austr...|2016-01-10|56.44|
| 10|1004.Asien--Austr...|2016-01-11| 56.0|
| 11|1004.Asien--Austr...|2016-01-12|56.72|
| 12|1004.Asien--Austr...|2016-01-13|56.11|
| 13|1004.Asien--Austr...|2016-01-14|55.08|
| 14|1004.Asien--Austr...|2016-01-15|56.97|
| 15|1004.Asien--Austr...|2016-01-16|56.97|
| 16|1004.Asien--Austr...|2016-01-17|56.97|
| 17|1004.Asien--Austr...|2016-01-18|54.69|
| 18|1004.Asien--Austr...|2016-01-19|56.33|
| 19|1004.Asien--Austr...|2016-0

In [21]:
start = time.time()

stocks_df = stocks_df.withColumn('price', f.col('price').cast(FloatType())).repartition('date')

print(time.time() - start)

0.10554790496826172


In [22]:
from pyspark.sql.types import FloatType, DoubleType

start = time.time()

big_df = stocks_df.alias('a').join(df.alias('b'), f.col('a.date') == f.col('b.date'), 'inner').repartition('stock_name', 'combo_id')
df.unpersist()
big_df = big_df.select('a.date', 'stock_name', 'combo_id', 'price', 'avg', 'min', 'max').cache()


print(time.time() - start)

0.33243608474731445


In [23]:
list(big_df.schema)

[StructField(date,StringType,true),
 StructField(stock_name,StringType,true),
 StructField(combo_id,StringType,true),
 StructField(price,FloatType,true),
 StructField(avg,FloatType,true),
 StructField(min,FloatType,true),
 StructField(max,FloatType,true)]

In [24]:
big_df.columns

['date', 'stock_name', 'combo_id', 'price', 'avg', 'min', 'max']

In [25]:
big_df.count()

3800000

----

In [26]:
##big_df.show()

----

#### This code cell is redundant 

from pyspark.sql.types import ArrayType, FloatType
import functools
def fudf(val):
    return functools.reduce(lambda x, y:x+y, val)

flattenUdf = f.udf(fudf, ArrayType(FloatType()))

-----

In [27]:
#Cache: 0.281
#0.100

In [28]:
start = time.time()

bigdf_grouped = big_df.groupBy(["stock_name", 'combo_id']).agg(f.collect_list("price").alias('price'),
                                                               f.collect_list("avg").alias('avg'),
                                                               f.collect_list("min").alias('min'),
                                                               f.collect_list("max").alias('max'))
                                                          #.select("stock_name",'combo_id',
                                                          #        flattenUdf("min").alias("min"),
                                                          #        flattenUdf("max").alias("max"),
                                                          #        flattenUdf("avg").alias("avg"),
                                                          #        flattenUdf("price").alias("price"))
big_df.unpersist()

print(time.time() - start)

0.19008922576904297


In [29]:
import numpy as np

def cosine(a, b):
  return float(np.dot(np.array(a), np.array(b)) / (np.linalg.norm(np.array(a)) * np.linalg.norm(np.array(b))))

In [30]:
cosine_udf = f.udf(cosine, FloatType())

----

In [31]:
#Cache: 0.246

In [32]:
start = time.time()

#bigdf_grouped.cache()

df_final = bigdf_grouped.select('stock_name', 'combo_id', cosine_udf('price', 'max').alias('price_max_cosine'),
                                                          cosine_udf('price', 'min').alias('price_min_cosine'),
                                                          cosine_udf('price', 'avg').alias('price_avg_cosine')).cache()
#bigdf_grouped.unpersist()

print(time.time() - start)

0.24666500091552734


In [33]:
#No cache: 365.07
#Cache: 24.694

In [34]:
start = time.time()

df_final.orderBy('price_avg_cosine', ascending=False).show()

df_final.unpersist()

print(time.time() - start)

+--------------------+--------------------+----------------+----------------+----------------+
|          stock_name|            combo_id|price_max_cosine|price_min_cosine|price_avg_cosine|
+--------------------+--------------------+----------------+----------------+----------------+
|10679.Asien--Aust...|06207099999_08524...|       0.9889081|       0.9046139|      0.96800274|
|10679.Asien--Aust...|03066099999_08524...|       0.9891048|       0.8502761|       0.9667747|
|10698.Asien--Aust...|06207099999_08524...|      0.98504055|       0.9026194|       0.9648411|
|10612.Asien--Aust...|06207099999_08524...|       0.9860405|      0.90060776|       0.9646363|
|10679.Asien--Aust...|03862099999_08524...|       0.9890492|       0.8536509|       0.9641612|
|1004.Asien--Austr...|06207099999_08524...|      0.97955555|      0.90912265|       0.9641316|
|10882.Asien--Aust...|06207099999_08524...|      0.98494434|      0.89949673|      0.96351963|
|10698.Asien--Aust...|03066099999_08524...|       

----

EVERYTHING BELOW IS REDUNDANT

----

In [ ]:
from pyspark.sql.functions import PandasUDFType, pandas_udf

In [ ]:
import numpy as np

# Names of columns 
a, b = "price", "avg" #('min', 'max', 'avg')
cosine_sim_col = "cosine_sim"

@pandas_udf(big_df.schema, PandasUDFType.GROUPED_MAP)
def cos_sim(df):
    df[cosine_sim_col] = float(np.dot(df[a], df[b]) / (np.linalg.norm(df[a]) * np.linalg.norm(df[b])))
    return df

In [ ]:
df_final = big_df.groupby(["stock_name", "combo_id"]).apply(cos_sim)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [ ]:
df_final.count()

3800000

In [ ]:
df_final.write.mode('overwrite').csv('/content/drive/MyDrive/BigDataManagement/final data/cosines.csv')

In [ ]:
df_final.groupBy(["stock_name", "combo_id"]).agg(f.avg("cosine_sim").alias("cossim")).orderBy('cossim', ascending=False).show()

+--------------------+--------------------+------------------+
|          stock_name|            combo_id|            cossim|
+--------------------+--------------------+------------------+
|10679.Asien--Aust...|06207099999_08524...|0.9680027365684509|
|10679.Asien--Aust...|03066099999_08524...|0.9667748808860779|
|10698.Asien--Aust...|06207099999_08524...|0.9648412466049194|
|10612.Asien--Aust...|06207099999_08524...| 0.964636504650116|
|10679.Asien--Aust...|03862099999_08524...|0.9641613364219666|
|1004.Asien--Austr...|06207099999_08524...|0.9641316533088684|
|10882.Asien--Aust...|06207099999_08524...|0.9635195732116699|
|10698.Asien--Aust...|03066099999_08524...|0.9632366299629211|
|1004.Asien--Austr...|03066099999_08524...|0.9627444744110107|
|10882.Asien--Aust...|03066099999_08524...|0.9626548290252686|
|10612.Asien--Aust...|03066099999_08524...|0.9624868631362915|
|10698.Asien--Aust...|03862099999_08524...| 0.961978554725647|
|10882.Asien--Aust...|03862099999_08524...|0.9614697694

In [ ]:
df_final.groupBy(["stock_name", "combo_id"]).agg(f.avg("cosine_sim").alias("cossim")).orderBy('cossim', ascending=False).count()

3800